In [0]:
%pip install splink==2.1.14

  Obtaining dependency information for splink==2.1.14 from https://files.pythonhosted.org/packages/e4/b1/a48d8214fd3fbbb7a62e8efcce50489e74e3769f7a48ea827875a6912ef2/splink-2.1.14-py3-none-any.whl.metadata
  Using cached splink-2.1.14-py3-none-any.whl.metadata (7.5 kB)
  Obtaining dependency information for jsonschema<4.0,>=3.2 from https://files.pythonhosted.org/packages/c5/8f/51e89ce52a085483359217bc72cdbf6e75ee595d5b1d4b5ade40c7e018b8/jsonschema-3.2.0-py2.py3-none-any.whl.metadata
  Using cached jsonschema-3.2.0-py2.py3-none-any.whl.metadata (7.8 kB)
  Obtaining dependency information for sqlglot==1.23.1 from https://files.pythonhosted.org/packages/3d/94/16b706fc1dbded4213b9335e281d5d5ac118a11718d6afef58a1a6ef2f67/sqlglot-1.23.1-py3-none-any.whl.metadata
  Using cached sqlglot-1.23.1-py3-none-any.whl.metadata (6.9 kB)
Using cached splink-2.1.14-py3-none-any.whl (1.3 MB)
Using cached sqlglot-1.23.1-py3-none-any.whl (37 kB)
Using cached jsonschema-3.2.0-py2.py3-none-any.whl (56 kB)
  

In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
def tear_down():
  import shutil
  try:
    shutil.rmtree(temp_directory)
  except:
    pass
  _ = sql("DROP DATABASE IF EXISTS {} CASCADE".format(database_name))
  dbutils.fs.rm(home_directory, True)

In [0]:
import re
from pathlib import Path

# We ensure that all objects created in that notebooks will be registered in a user specific database. 
useremail = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()
username = useremail.split('@')[0]

# Similar to database, we will store actual content on a given path
home_directory = '/Workspace/Users/trishaldoshi@outlook.com/aml'
dbutils.fs.mkdirs(home_directory)

# Please replace this cell should you want to store data somewhere else.
database_name = '{}_aml'.format(re.sub('\W', '_', username))

# Where we might store temporary data on local disk
temp_directory = "/Workspace/Users/trishaldoshi@outlook.com/aml_temp'

In [0]:
display(home_directory)

'/Workspace/Users/trishaldoshi@outlook.com/aml'

In [0]:
tear_down()

In [0]:
display(database_name)

'trishaldoshi_aml'

In [0]:
%sql 
CREATE DATABASE IF NOT EXISTS trishaldoshi_aml

In [0]:
import re

config = {
  'db_transactions': f"{database_name}.transactions",
  'db_entities': f"{database_name}.entities",
  'db_dedupe': f"{database_name}.dedupe",
  'db_synth_scores': f"{database_name}.synth_scores",
  'db_structuring': f"{database_name}.structuring",
  'db_structuring_levels': f"{database_name}.structuring_levels",
  'db_roundtrips': f"{database_name}.roundtrips",
  'db_risk_propagation': f"{database_name}.risk_propagation",
  'db_streetview': f"{database_name}.streetview",
  'db_dedupe_records': f"{database_name}.dedupe_splink",
}

In [0]:
tables = set(sql("SHOW TABLES IN {}".format(database_name)).toPandas().set_index("tableName").index)

if len(tables) == 0:
  
  print("Creating input tables {} and {}".format(config['db_transactions'], config['db_entities']))
  # populate table with sample records
  spark \
    .read \
    .format("parquet") \
    .load("s3://db-gtm-industry-solutions/data/fsi/aml_introduction/txns.parquet") \
    .write \
    .saveAsTable(config['db_transactions'])
  
  spark \
    .read \
    .format("parquet") \
    .load("s3://db-gtm-industry-solutions/data/fsi/aml_introduction/entities.parquet") \
    .write \
    .saveAsTable(config['db_entities'])

  spark \
    .read \
    .format("csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load("s3://db-gtm-industry-solutions/data/fsi/aml_introduction/dedupe.csv") \
    .write \
    .saveAsTable(config['db_dedupe'])

Creating input tables trishaldoshi_aml.transactions and trishaldoshi_aml.entities


In [0]:
import mlflow
experiment_name = f"/Users/{useremail}/aml_experiment"
mlflow.set_experiment(experiment_name) 

2025/03/03 13:08:16 INFO mlflow.tracking.fluent: Experiment with name '/Users/trishaldoshi@outlook.com/aml_experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/1251187827433329', creation_time=1741007296733, experiment_id='1251187827433329', last_update_time=1741007296733, lifecycle_stage='active', name='/Users/trishaldoshi@outlook.com/aml_experiment', tags={'mlflow.experiment.sourceName': '/Users/trishaldoshi@outlook.com/aml_experiment',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'trishaldoshi@outlook.com',
 'mlflow.ownerId': '4503932489037577'}>